<a href="https://colab.research.google.com/github/Aravindh4404/FYPSeagullClassification01/blob/main/VGG2optuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import necessary libraries
!pip install optuna
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime
from tqdm import tqdm  # For progress bars

# Install Optuna if not already installed
# !pip install optuna

import optuna
from optuna.trial import TrialState

# Set random seeds for reproducibility
def set_seed(seed=42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# Define the device for computation
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Mount Google Drive to save and load the model (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Define the folder to save model checkpoints
date_str = datetime.now().strftime('%Y%m%d')
checkpoint_folder = f'/content/drive/My Drive/FYP/VGGModel/HQ2ltst_{date_str}/'
os.makedirs(checkpoint_folder, exist_ok=True)

# Data Augmentation for Training Set
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG expects 224x224 input size
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),  # ImageNet normalization
])

# Simple resizing for validation and test sets
transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])

# Load datasets
data_path = '/content/drive/My Drive/FYP/Dataset/HQ2/train'
test_data_path = '/content/drive/My Drive/FYP/Dataset/HQ2/test'
train_dataset = datasets.ImageFolder(data_path, transform=transform_train)
test_dataset = datasets.ImageFolder(test_data_path, transform=transform_val_test)

# Split the dataset into 80% training and 20% validation
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_subset, val_subset = random_split(train_dataset, [train_size, val_size],
                                       generator=torch.Generator().manual_seed(42))  # Ensure reproducibility

# Create data loaders
batch_size = 32
num_workers = 4  # Adjust based on your system

train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Define class names as per dataset
class_names = train_dataset.classes  # Automatically get class names from ImageFolder
num_classes = len(class_names)
print(f"Classes: {class_names}")

# Use Pre-trained VGG-16 model and modify it for binary classification
class VGG16Modified(nn.Module):
    def __init__(self, num_classes=2):
        super(VGG16Modified, self).__init__()
        self.vgg = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
        # Replace the classifier with a custom binary classification layer
        num_ftrs = self.vgg.classifier[6].in_features
        self.vgg.classifier[6] = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, num_classes)
        )

    def forward(self, x):
        return self.vgg(x)

# Define the validation loop returning accuracy
def validate(model, loader, criterion):
    model.eval()
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    average_loss = val_loss / len(loader)
    print(f'Validation Loss: {average_loss:.4f}, Validation Accuracy: {accuracy:.2f}%')
    return accuracy, average_loss

# Define the objective function for Optuna
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-6, 1e-3)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.3, 0.7)
    optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'AdamW', 'SGD'])
    momentum = 0.9  # Default momentum for SGD

    # Initialize the model
    model = VGG16Modified(num_classes=num_classes).to(device)

    # Modify dropout rate if necessary
    # Note: In the current model, dropout is fixed at 0.5. To make it tunable, you need to adjust the model definition.
    # For simplicity, we'll proceed with the existing dropout.

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    # Define optimizer
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:  # SGD
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum, weight_decay=weight_decay)

    # Define scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=False)

    # Training parameters
    epochs = 20
    best_val_acc = 0.0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)  # Gradient clipping
            optimizer.step()

            running_loss += loss.item()

        average_train_loss = running_loss / len(train_loader)
        print(f"Trial {trial.number}, Epoch {epoch+1}/{epochs}, Train Loss: {average_train_loss:.4f}")

        # Validate the model
        val_acc, val_loss = validate(model, val_loader, criterion)

        # Update the scheduler
        scheduler.step(val_acc)

        # Report intermediate objective value
        trial.report(val_acc, epoch)

        # Handle pruning based on the intermediate value
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

        # Save the best validation accuracy for this trial
        if val_acc > best_val_acc:
            best_val_acc = val_acc

    return best_val_acc

# Create the study
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=20, timeout=None)

# Print study statistics
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print(f"  Validation Accuracy: {trial.value:.2f}%")
print("  Best hyperparameters: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# Save the study
study_dir = os.path.join(checkpoint_folder, 'optuna_study')
os.makedirs(study_dir, exist_ok=True)
joblib.dump(study, os.path.join(study_dir, 'study.pkl'))
print(f"Optuna study saved at {study_dir}")

# Function to plot the optimization history
def plot_optimization_history(study):
    optuna.visualization.plot_optimization_history(study)
    plt.show()

# Function to plot the parameter importances
def plot_param_importances(study):
    optuna.visualization.plot_param_importances(study)
    plt.show()

# Plot optimization history
plot_optimization_history(study)

# Plot parameter importances
plot_param_importances(study)

# Retrain the model with the best hyperparameters
def train_with_best_params(best_params):
    learning_rate = best_params['learning_rate']
    weight_decay = best_params['weight_decay']
    optimizer_name = best_params['optimizer']
    # dropout_rate = best_params.get('dropout_rate', 0.5)  # Not used in current model

    # Initialize the model
    model = VGG16Modified(num_classes=num_classes).to(device)

    # Define loss function
    criterion = nn.CrossEntropyLoss()

    # Define optimizer
    if optimizer_name == 'Adam':
        optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    elif optimizer_name == 'AdamW':
        optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:  # SGD
        optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=weight_decay)

    # Define scheduler
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=5, verbose=True)

    # Training parameters
    epochs = 20
    best_val_acc = 0.0
    best_model_path = os.path.join(checkpoint_folder, f"best_model_vgg_{date_str}.pth")

    train_losses = []
    val_accuracies = []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0)  # Gradient clipping
            optimizer.step()

            running_loss += loss.item()

        average_train_loss = running_loss / len(train_loader)
        train_losses.append(average_train_loss)
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {average_train_loss:.6f}")

        # Validate the model
        val_acc, val_loss = validate(model, val_loader, criterion)
        val_accuracies.append(val_acc)

        # Update the scheduler
        scheduler.step(val_acc)

        # Save the best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), best_model_path)
            print(f"New best model saved with accuracy: {best_val_acc:.2f}% at {best_model_path}")

    # Plot training and validation metrics
    epochs_range = range(1, epochs + 1)

    plt.figure(figsize=(12, 5))

    # Plot Loss
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, train_losses, label='Training Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss')
    plt.legend()
    plt.grid(True)

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, val_accuracies, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Validation Accuracy')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

    # Save the final model
    final_model_path = os.path.join(checkpoint_folder, f"final_model_vgg_{date_str}.pth")
    torch.save(model.state_dict(), final_model_path)
    print(f"Final model saved at {final_model_path}")

    return model

# Retrain the model with the best hyperparameters
best_params = study.best_trial.params
print("Retraining the model with the best hyperparameters...")
trained_model = train_with_best_params(best_params)

# Function to evaluate the model on the test set
def test(model, loader, criterion):
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    average_test_loss = test_loss / len(loader)
    accuracy = 100 * correct / total
    print(f'Test Loss: {average_test_loss:.6f}, Test Accuracy: {accuracy:.2f}%')
    return average_test_loss, accuracy

# Evaluate the trained model on the test set
print("Evaluating the trained model on the test set...")
test_loss, test_accuracy = test(trained_model, test_loader, nn.CrossEntropyLoss())

# Save the trained model
final_model_path = os.path.join(checkpoint_folder, f"trained_model_vgg_{date_str}.pth")
torch.save(trained_model.state_dict(), final_model_path)
print(f"Trained model saved at {final_model_path}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00
Using device: cpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
[I 2024-12-11 09:44:00,604] A new study created in memory with name: no-name-3e450dcf-0516-4503-8898-84b259122eb4
<ipython-input-5-a6658be12d16>:127: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-3)
<ipython-input-5-a6658be12d16>:128: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will

Classes: ['Glaucous_Winged_Gull', 'Slaty_Backed_Gull']


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Trial 0, Epoch 1/20, Train Loss: 0.3761
Validation Loss: 0.1247, Validation Accuracy: 95.45%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
